# Rendering with Blender
This predictor demonstrates using Function as a serverless 3D rendering service. To do so, we use the open-source Blender program.

## Dependencies
First, we need to install system library dependencies:

In [ ]:
# Install Blender system library dependencies
!apt-get install build-essential subversion cmake libx11-dev                        \
    libsm-dev libxxf86vm-dev libxcursor-dev libxi-dev libxrandr-dev                 \
    libxinerama-dev libegl-dev libwayland-dev wayland-protocols libxkbcommon-dev    \
    libdbus-1-dev linux-libc-dev -y

Now, let's install Blender for Python:

In [ ]:
# Install Blender for Python
%pip install bpy

## Implementation

In [7]:
from PIL import Image
from tempfile import mkstemp

def predict () -> Image.Image:
    """
    Render the default cube scene in Blender and return the image.

    Returns:
        PIL.Image: Result image.
    """
    # Load the default scene
    from bpy import context, data, ops
    ops.wm.read_homefile()    
    # Configure Cycles renderer to use all available GPUs
    context.scene.render.engine = "CYCLES"
    cycles_settings = context.scene.cycles
    cycles_settings.samples = 16
    context.scene.cycles.device = "GPU"
    context.preferences.addons["cycles"].preferences.compute_device_type = "OPTIX"
    context.preferences.addons["cycles"].preferences.get_devices()
    for device in context.preferences.addons["cycles"].preferences.devices:
        device["use"] = True
    # Render to image
    _, render_path = mkstemp(suffix=".png")
    context.scene.render.filepath = render_path
    ops.render.render(write_still=True)
    # Load rendered image
    result = Image.open(render_path)
    # Return
    return result

## Provisioning
We want our renderer to be GPU-accelerated, so we'll use the `A40` acceleration. Open a terminal and run the following command:
```bash
# Create the predictor on Function and use GPU acceleration
fxn create @username/rendering rendering.ipynb --acceleration A40
```

> Replace `username` with your Function username.